In [1]:
#import pixiedust
%matplotlib widget


In [2]:
import backtrader as bt
from backtradermql5.mt5store import MTraderStore
from datetime import datetime, timedelta

In [3]:
class SuperTrend(bt.Indicator):
    """
    SuperTrend Algorithm :
    
        BASIC UPPERBAND = (high + low) / 2 + Multiplier * ATR
        BASIC lowERBAND = (high + low) / 2 - Multiplier * ATR
        
        FINAL UPPERBAND = IF( (Current BASICUPPERBAND < Previous FINAL UPPERBAND) or (Previous close > Previous FINAL UPPERBAND))
                            THEN (Current BASIC UPPERBAND) ELSE Previous FINALUPPERBAND)
        FINAL lowERBAND = IF( (Current BASIC lowERBAND > Previous FINAL lowERBAND) or (Previous close < Previous FINAL lowERBAND)) 
                            THEN (Current BASIC lowERBAND) ELSE Previous FINAL lowERBAND)
        SUPERTREND = IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current close <= Current FINAL UPPERBAND)) THEN
                        Current FINAL UPPERBAND
                    ELSE
                        IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current close > Current FINAL UPPERBAND)) THEN
                            Current FINAL lowERBAND
                        ELSE
                            IF((Previous SUPERTREND = Previous FINAL lowERBAND) and (Current close >= Current FINAL lowERBAND)) THEN
                                Current FINAL lowERBAND
                            ELSE
                                IF((Previous SUPERTREND = Previous FINAL lowERBAND) and (Current close < Current FINAL lowERBAND)) THEN
                                    Current FINAL UPPERBAND
        
    """

    lines = ('super_trend', )
    params = (
        ('period', 7),
        ('multiplier', 3),
    )
    plotlines = dict(super_trend=dict(_name='ST', color='blue', alpha=1))

    plotinfo = dict(subplot=False)

    def __init__(self):
        self.st = [0]
        self.finalupband = [0]
        self.finallowband = [0]
        self.addminperiod(self.p.period)
        atr = bt.ind.ATR(self.data, period=self.p.period)
        self.upperband = (self.data.high +
                          self.data.low) / 2 + self.p.multiplier * atr
        self.lowerband = (self.data.high +
                          self.data.low) / 2 - self.p.multiplier * atr

    def next(self):
        pre_upband = self.finalupband[0]
        pre_lowband = self.finallowband[0]

        if self.upperband[0] < self.finalupband[-1] or self.data.close[
                -1] > self.finalupband[-1]:
            self.finalupband[0] = self.upperband[0]

        else:
            self.finalupband[0] = self.finalupband[-1]

        if self.lowerband[0] > self.finallowband[-1] or self.data.close[
                -1] < self.finallowband[-1]:

            self.finallowband[0] = self.lowerband[0]

        else:
            self.finallowband[0] = self.finallowband[-1]

        if self.data.close[0] <= self.finalupband[0] and (
            (self.st[-1] == pre_upband)):

            self.st[0] = self.finalupband[0]
            self.lines.super_trend[0] = self.finalupband[0]

        elif (self.st[-1] == pre_upband) and (self.data.close[0] >
                                              self.finalupband[0]):

            self.st[0] = self.finallowband[0]
            self.lines.super_trend[0] = self.finallowband[0]

        elif (self.st[-1] == pre_lowband) and (self.data.close[0] >=
                                               self.finallowband[0]):

            self.st[0] = self.finallowband[0]
            self.lines.super_trend[0] = self.finallowband[0]

        elif (self.st[-1] == pre_lowband) and (self.data.close[0] <
                                               self.finallowband[0]):

            self.st[0] = self.finalupband[0]
            self.lines.super_trend[0] = self.st[0]

In [4]:
#%%pixie_debugger

class MyStrategy(bt.Strategy):
    params = (('ma1period', 30), ('ma2period', 5), ('ma3period', 30),
              ('atrperiod', 6), ('stop_loss', 0.08), )

    def log(self, txt, dt=None, doprint=True):
        ''' Logging function for this strategy'''
        if doprint:
            dt = dt or self.datas[0].datetime.datetime()
            print(f'{dt}: {txt}')

    def __init__(self):
        self.dataclose = self.datas[0].close
        self.order = None
        self.buyprice = None
        self.buycomm = None
        self.inBuyPosition = False
        self.inSellPosition = False

        #super().__init__()
        #self.broker.set_coc(True)

        # indicators
        self.ma1 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.ma1period)
        self.ma2 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.ma2period)
        self.ma3 = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.ma3period)

        #
        self.atr = bt.ind.AverageTrueRange(period=self.params.atrperiod,
                                           plot=False)
        #self.x = SuperTrend(plot=True)

    def notify_order(self, order):

        if order.status in [order.Submitted, order.Accepted]:
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log('BUY EXECUTED, Price: %.5f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price, order.executed.value,
                          order.executed.comm))
                # set stop price
                #stop_price = order.executed.price * (1.0 - self.p.stop_loss)
                #self.sell(exectype=bt.Order.Stop, price=stop_price)

            elif order.issell():
                self.log('SELL EXECUTED, Price: %.5f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price, order.executed.value,
                          order.executed.comm))
                # set stop price
                #stop_price = order.executed.price * (1.0 + self.p.stop_loss)
                #self.buy(exectype=bt.Order.Stop, price=stop_price)

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        self.log('Close, %.5f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:
            # Not yet ... we MIGHT BUY if
            # sma1 > sma2 > sma3

            if self.ma1[0] > self.ma2[0] > self.ma3[0]:
                # also check if current close is higher than previous one
                if self.dataclose[0] > self.dataclose[-1]:
                    # BUY!!! (with all possible default parameters)
                    self.log('BUY CREATE1, %.5f' % self.dataclose[0])

                    self.order = self.buy()
                    #self.buy_order = self.buy_bracket(
                    #stop_price=self.dataclose[0] * 0.98
                    #self.order = self.buy_bracket(
                    #    stopprice=stop_price)
                    
                    self.inBuyPosition = True

            if self.ma1[0] < self.ma2[0] < self.ma3[0]:
                # also check if current close is lower than previous one
                if self.dataclose[0] < self.dataclose[-1]:
                    # SELL, SELL, SELL!!! (with all possible default parameters)
                    self.log('SELL CREATE1, %.5f' % self.dataclose[0])
                    
                    self.order = self.sell()
                    #stop_price=self.dataclose[0] * 1.02
                    #self.order = self.sell_bracket(
                    #    stopprice=stop_price)

                    self.inSellPosition = True

        else:
            # Already in the market ..
            # we will buy if existing position is sell

            if self.inBuyPosition == True:
                #if self.ma1[0] < self.ma2[0] < self.ma3[0]:
                if not (self.ma1[0] > self.ma2[0] > self.ma3[0]):
                    # SELL, SELL, SELL!!! (with all possible default parameters)
                    self.log('SELL CREATE2, %.5f' % self.dataclose[0])
                    #self.order = self.sell()
                    self.order = self.close()
                    #self.inSellPosition = True
                    self.inBuyPosition = False

                    #stop_price = self.data.close[0] * (1.0 + self.p.stop_loss)
                    #self.buy(exectype=bt.Order.Stop, price=stop_price)

            if self.inSellPosition == True:
                #if self.ma1[0] > self.ma2[0] > self.ma3[0]:
                if not (self.ma1[0] < self.ma2[0] < self.ma3[0]):
                    # BUY!!! (with all possible default parameters)
                    self.log('BUY CREATE2, %.5f' % self.dataclose[0])
                    #self.order = self.buy()
                    self.order = self.close()
                    #self.inBuyPosition = True
                    self.inSellPosition = False

                    #stop_price = self.data.close[0] * (1.0 - self.p.stop_loss)
                    #self.sell(exectype=bt.Order.Stop, price=stop_price)

    def stop(self):
        self.log('(SMA period %2d, %2d, %2d) Ending Value %.2f' %
                 (self.params.ma1period, self.params.ma2period,
                  self.params.ma3period, self.broker.getvalue()),
                 doprint=True)

In [5]:
class superTrendStrategy(bt.Strategy):
    def log(self, txt, dt=None):
        if True:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s - %s' % (dt.isoformat(), txt))

    def __init__(self):
        #self.x = SuperTrend(self.data)
        self.x = SuperTrend(plot=True)
        self.dclose = self.datas[0].close
        self.cross = bt.ind.CrossOver(self.dclose, self.x)

    def notify(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return
        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED: %s, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.data._name, order.executed.price,
                     order.executed.value, order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
                self.opsize = order.executed.size
            else:  # Sell
                self.log(
                    'SELL EXECUTED: %s, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.data._name, order.executed.price,
                     order.executed.value, order.executed.comm))

    def notify_trade(self, trade):
        if trade.isclosed:
            self.log('TRADE PROFIT: EQ %s, GROSS %.2f, NET %.2f' %
                     ('Closed', trade.pnl, trade.pnlcomm))
        elif trade.justopened:
            self.log('TRADE OPENED: EQ %s, SIZE %2d' % ('Opened', trade.size))

    def next(self):
        pos = self.getposition(self.data)
        dpos = pos.size
        if self.cross[0] == 1 and dpos <= 0:
            self.order_target_percent(data=self.data, target=1)
        elif self.cross[0] == -1 and dpos >= 0:
            self.order_target_percent(data=self.data, target=-1)


In [6]:
def runstrat(ppair='EURUSD',
             ptf=bt.TimeFrame.Minutes,
             pcomp=1,
             ptf1=bt.TimeFrame.Minutes,
             pcomp1=0,
             pstart_date="2020-03-21",
             pend_date="",
             phost='192.168.100.110',
             pdebug=False,
             phistory=True,
             pmt5broker=False,
             psizer_type=bt.sizers.FixedSize,
             pcash=10000,
             pstake=1,
             pcommission=15,
             pmargin=1000,
             pmult=100000,
             pplot=True,
             panalyze=True,
             pstrategy=MyStrategy,
             pma1period=5,
             pma2period=10,
             pma3period=15,
             patrperiod=20):
#             pdoprint=False):

    cerebro = bt.Cerebro()
    store = MTraderStore(host=phost, debug=pdebug)

    # get data
    data = store.getdata(dataname=ppair,
                        timeframe=ptf,
                        compression=pcomp,
                        fromdate = pstart_date,
                        todate   = pend_date,
                        historical=phistory
                        )
    
    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # resample the data if required
    if pcomp1:
        cerebro.resampledata(data, timeframe=ptf1, compression=pcomp1)

    # Set our desired cash start
    cerebro.broker.setcash(pcash)

    # set sizer
    cerebro.addsizer(psizer_type, stake=pstake)

    #----- Normal Strategy
    #cerebro.addstrategy(MyStrategy, ma1period=5, ma2period=10, ma3period=17, atrperiod=10)
    #    pstrategy,
    cerebro.addstrategy(pstrategy ,
                        ma1period=pma1period,
                        ma2period=pma2period,
                        ma3period=pma3period,
                        atrperiod=patrperiod)
                        #doprint=pdoprint)

    #----- Set commission
    #cerebro.broker.setcommission(commission=15,margin=1000, mult=100000)
    cerebro.broker.setcommission(commission=pcommission,
                                 margin=pmargin,
                                 mult=pmult)

    # test analyzer
    if panalyze:
        cerebro.addanalyzer(bt.analyzers.TimeReturn, timeframe=bt.TimeFrame.Years)
        cerebro.addanalyzer(bt.analyzers.SharpeRatio, timeframe=bt.TimeFrame.Years)
        cerebro.addanalyzer(bt.analyzers.SQN, )

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    #results = cerebro.run(maxcpus=1, stdstats=False)
    results = cerebro.run()
    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    if pplot:
        cerebro.plot(style='candlestick', volume=True)


    if panalyze:
        strat = results[0]
        # Results of own analyzers
        al = strat.analyzers.timereturn
        print('-- Time Return:')
        for k, v in al.get_analysis().items():
            print('{}: {}'.format(k, v))

        al = strat.analyzers.sharperatio
        print('-- Sharpe Ratio:')
        for k, v in al.get_analysis().items():
            print('{}: {}'.format(k, v))

        al = strat.analyzers.sqn
        print('-- SQN:')
        for k, v in al.get_analysis().items():
            print('{}: {}'.format(k, v))

In [7]:
    
runstrat(ppair='EURUSD',
         ptf=bt.TimeFrame.Minutes,
         pcomp=240,
         ptf1=bt.TimeFrame.Minutes,
         pcomp1=0,
         pstart_date=datetime.now() - timedelta(days=58),
         pend_date=datetime.now() - timedelta(days=40),
         phost='192.168.100.113',
         pdebug=False,
         phistory=True,
         pmt5broker=False,
         psizer_type=bt.sizers.FixedSize,
         pcash=10000,
         pstake=1,
         pcommission=15,
         pmargin=1000,
         pmult=100000,
         pplot=True,
         panalyze=False,
         pstrategy=MyStrategy,
         pma1period=5,
         pma2period=9,
         pma3period=15,
         patrperiod=1) 
         #pstrategy=superTrendStrategy,
         #pdoprint=True) # this also doesn't work now.. fix this..
         #pstrategy=MyStrategy, (don't forget to enable the xxxperiod param in addStrategy, FIX this!)


Starting Portfolio Value: 10000.00


NotDone: Data socket timeout ERROR